In [23]:
# data
import pandas as pd
import numpy as np
import gc
from typing import List

# models
from lightgbm import LGBMClassifier, log_evaluation
from sklearn.model_selection import StratifiedGroupKFold

import warnings
warnings.simplefilter(action = "ignore", category = RuntimeWarning)


def get_train_data(TRAIN_PATH: str) -> pd.DataFrame:
    '''Returns train dataset'''
    df_train = pd.read_parquet(TRAIN_PATH)
    return df_train


def get_test_data(TEST_PATH: str) -> pd.DataFrame:
    '''Returns test dataset'''
    df_test = pd.read_parquet(TEST_PATH)
    return df_test


def get_target(TARGET_PATH: str) -> pd.DataFrame:
    '''Retruns dataset with train targets'''
    df_train_target = pd.read_csv(TARGET_PATH)
    return df_train_target


def get_train_data_with_target_merged(df_train: pd.DataFrame, df_train_target: pd.DataFrame) -> pd.DataFrame:
    '''Retruns train dataset with target variable merged'''
    df_train_w_target = (
        df_train
        .merge(df_train_target,
            on='customer_ID',
            how='left'
        )
    )
    # df_train_w_target.groupby('target', dropna=False).count()['customer_ID']
    '''
    target
    0    4153582
    1    1377869
    Name: customer_ID, dtype: int64    
    '''
    return df_train_w_target


def get_all_features(df: pd.DataFrame) -> List:
    '''Returns list of all features from the dataset'''
    return list(df)


def get_cat_features() -> List:
    '''Returns list of categorical features from the dataset'''
    cat_features = ['B_30', 'B_38', 'D_114', 
                    'D_116', 'D_117', 'D_120', 
                    'D_126', 'D_63', 'D_64', 
                    'D_66', 'D_68']
    
    return cat_features


def get_num_features(all_features: List, cat_features: List) -> List:
    '''Returns list of all numerical features from the dataset'''
    num_feats = [col for col in all_features if col not in cat_features + ['customer_ID', 'S_2', 'target']]

    return num_feats


def get_df_w_aggrs(df: pd.DataFrame, feats: List) ->  pd.DataFrame:
    '''Returns dataframe with generated aggregates based on numerical features'''

    cid = pd.Categorical(df.pop('customer_ID'), ordered=True)
    last = (cid != np.roll(cid, -1)) # mask for last statement of every customer

    df_min = (df
        .groupby(cid)
        .min()[feats]
        .rename(columns={f: f"{f}_min" for f in feats})
    )
    print(df_min.shape)

    df_max = (df
        .groupby(cid)
        .max()[feats]
        .rename(columns={f: f"{f}_max" for f in feats})
    )
    print(df_max.shape)

    df_avg = (df
        .drop('S_2', axis='columns')
        .groupby(cid)
        .mean()[feats]
        .rename(columns={f: f"{f}_avg" for f in feats})
    )
    print(df_avg.shape)

    df_last = (df
        .loc[last, feats]
        .rename(columns={f: f"{f}_last" for f in feats})
        .set_index(np.asarray(cid[last]))
    )
    print(df_last.shape)

    df_aggrs = (pd.concat([df_min, df_max, df_avg, df_last], axis=1)
        .reset_index()
        .rename(columns={'index': 'customer_ID'})
    )
    print(df_aggrs.shape)

    '''
    del df, df_min, df_max, df_avg, cid, last
    gc.collect()
    '''
    return df_aggrs


def check_zapolnenie(df: pd.DataFrame) -> pd.DataFrame:
    '''Returns pd.DataFrame with isNotNullShare of each column of given df'''
    # Calculate percent of not null share each column 
    col_pct_notNull = [] 
    for col in df.columns: 
        percent_notNull = np.mean(~df[col].isnull())*100 
        col_pct_notNull.append([col, percent_notNull]) 
        
    col_pct_notNull_df = pd.DataFrame(col_pct_notNull, columns = ['column_name','isNotNullShare']).sort_values(by = 'isNotNullShare', ascending = False) 
    #print(col_pct_notNull_df)
    return col_pct_notNull_df

In [24]:
df_train = get_train_data(TRAIN_PATH='./data/train.parquet')

all_features = get_all_features(df_train)
cat_features = get_cat_features()
num_features = get_num_features(all_features, cat_features)
# len(all_features), len(cat_features), len(num_features) -> (190, 11, 178)

df_train_agg = get_df_w_aggrs(df=df_train, feats=num_features+cat_features)
df_train_target = get_target(TARGET_PATH='./data/train_labels.csv')
df_train = get_train_data_with_target_merged(df_train=df_train_agg, df_train_target=df_train_target)

'''
df_train.target.value_counts()
target
0    340085
1    118828
Name: count, dtype: int64
'''

# df_test = get_test_data(TEST_PATH='./data/test.parquet')
# df_test = get_df_w_aggrs(df=df_test, feats=num_features+cat_features)

# zapolnenie_train = check_zapolnenie(df_train)
# zapolnenie_test = check_zapolnenie(df_test)

C:\Users\oobur\AppData\Local\Temp\ipykernel_14784\2036179205.py:81: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(cid)


(458913, 188)


C:\Users\oobur\AppData\Local\Temp\ipykernel_14784\2036179205.py:88: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(cid)


(458913, 188)


C:\Users\oobur\AppData\Local\Temp\ipykernel_14784\2036179205.py:96: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(cid)


(458913, 188)
(458913, 188)
(458913, 753)


'\ndf_train.target.value_counts()\ntarget\n0    340085\n1    118828\nName: count, dtype: int64\n'

In [25]:
df_train.shape

(458913, 754)

In [9]:
cat_features

['B_30',
 'B_38',
 'D_114',
 'D_116',
 'D_117',
 'D_120',
 'D_126',
 'D_63',
 'D_64',
 'D_66',
 'D_68']

In [9]:
clf = LGBMClassifier(n_estimators=200,
                     max_depth=2,
                     learning_rate=0.05,
                     colsample_bytree=0.8,
                     subsample=0.95,
                     subsample_freq=1,
                     random_state=42)

In [ ]:
'''
clf.fit(
    X: Union[lightgbm.compat.dt_DataTable, List[Union[List[float], List[int]]], numpy.ndarray, pandas.core.frame.DataFrame, scipy.sparse._matrix.spmatrix],
    y: Union[List[float], List[int], numpy.ndarray, pandas.core.series.Series, pandas.core.frame.DataFrame, pyarrow.lib.Array, pyarrow.lib.ChunkedArray],
    sample_weight: Union[List[float], List[int], numpy.ndarray, pandas.core.series.Series, pyarrow.lib.Array, pyarrow.lib.ChunkedArray, NoneType] = None,
    init_score: Union[List[float], List[List[float]], numpy.ndarray, pandas.core.series.Series, pandas.core.frame.DataFrame, pyarrow.lib.Table, pyarrow.lib.Array, pyarrow.lib.ChunkedArray, NoneType] = None,
    eval_set: Optional[List[Tuple[Union[lightgbm.compat.dt_DataTable, List[Union[List[float], List[int]]], numpy.ndarray, pandas.core.frame.DataFrame, scipy.sparse._matrix.spmatrix], Union[List[float], List[int], numpy.ndarray, pandas.core.series.Series, pandas.core.frame.DataFrame, pyarrow.lib.Array, pyarrow.lib.ChunkedArray]]]] = None,
    eval_names: Optional[List[str]] = None,
    eval_sample_weight: Optional[List[Union[List[float], List[int], numpy.ndarray, pandas.core.series.Series, pyarrow.lib.Array, pyarrow.lib.ChunkedArray]]] = None,
    eval_class_weight: Optional[List[float]] = None,
    eval_init_score: Optional[List[Union[List[float], List[List[float]], numpy.ndarray, pandas.core.series.Series, pandas.core.frame.DataFrame, pyarrow.lib.Table, pyarrow.lib.Array, pyarrow.lib.ChunkedArray]]] = None,
    eval_metric: Union[str, Callable[[Optional[numpy.ndarray], numpy.ndarray], Tuple[str, float, bool]], Callable[[Optional[numpy.ndarray], numpy.ndarray], List[Tuple[str, float, bool]]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray]], Tuple[str, float, bool]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray]], List[Tuple[str, float, bool]]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray], Optional[numpy.ndarray]], Tuple[str, float, bool]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray], Optional[numpy.ndarray]], List[Tuple[str, float, bool]]], List[Union[str, Callable[[Optional[numpy.ndarray], numpy.ndarray], Tuple[str, float, bool]], Callable[[Optional[numpy.ndarray], numpy.ndarray], List[Tuple[str, float, bool]]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray]], Tuple[str, float, bool]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray]], List[Tuple[str, float, bool]]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray], Optional[numpy.ndarray]], Tuple[str, float, bool]], Callable[[Optional[numpy.ndarray], numpy.ndarray, Optional[numpy.ndarray], Optional[numpy.ndarray]], List[Tuple[str, float, bool]]]]], NoneType] = None,
    feature_name: Union[List[str], ForwardRef("Literal['auto']")] = 'auto',
    categorical_feature: Union[List[str], List[int], ForwardRef("Literal['auto']")] = 'auto',
    callbacks: Optional[List[Callable]] = None,
    init_model: Union[str, pathlib.Path, lightgbm.basic.Booster, lightgbm.sklearn.LGBMModel, NoneType] = None,
) -> 'LGBMClassifier'
'''

In [27]:
df_train[['B_30_last']].value_counts()

B_30_last
 0           383087
 1            71127
 2             4668
-1               31
Name: count, dtype: int64

In [28]:
df_train[['target']]

,target
0,0
1,0
2,0
3,0
4,0
...,...
458908,0
458909,0
458910,0
458911,1


In [29]:
clf.fit(X=df_train[['B_30_last']],
        y=df_train[['target']],
        categorical_feature=['B_30_last'])

C:\Users\oobur\Projects\spbu_master\sem4\vkr\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\oobur\Projects\spbu_master\sem4\vkr\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Info] Number of positive: 118828, number of negative: 340085
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002016 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4
[LightGBM] [Info] Number of data points in the train set: 458913, number of used features: 1
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258934 -> initscore=-1.051519
[LightGBM] [Info] Start training from score -1.051519
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

LGBMClassifier(colsample_bytree=0.8, learning_rate=0.05, max_depth=2,
               n_estimators=200, random_state=42, subsample=0.95,
               subsample_freq=1)

In [ ]:
# отсутствует поддержка категориальных признаков

In [ ]:
kf = StratifiedKFold(n_splits=5)
for fold, (idx_tr, idx_va) in enumerate(kf.split(train, target)):
    X_tr, X_va, y_tr, y_va, model = None, None, None, None, None
    start_time = datetime.datetime.now()
    X_tr = train.iloc[idx_tr][features]
    X_va = train.iloc[idx_va][features]
    y_tr = target[idx_tr]
    y_va = target[idx_va]
    
    model = my_booster()
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', category=UserWarning)
        model.fit(X_tr, y_tr,
                  eval_set = [(X_va, y_va)], 
                  eval_metric=[lgb_amex_metric],
                  callbacks=[log_evaluation(100)])
    X_tr, y_tr = None, None
    y_va_pred = model.predict_proba(X_va, raw_score=True)
    score = amex_metric(y_va, y_va_pred)
    n_trees = model.best_iteration_
    if n_trees is None: n_trees = model.n_estimators
    print(f"{Fore.GREEN}{Style.BRIGHT}Fold {fold} | {str(datetime.datetime.now() - start_time)[-12:-7]} |"
          f" {n_trees:5} trees |"
          f"                Score = {score:.5f}{Style.RESET_ALL}")
    score_list.append(score)
    
    if INFERENCE:
        y_pred_list.append(model.predict_proba(test[features], raw_score=True))
        
    if ONLY_FIRST_FOLD: break # we only want the first fold
    
print(f"{Fore.GREEN}{Style.BRIGHT}OOF Score:                       {np.mean(score_list):.5f}{Style.RESET_ALL}")

In [4]:
def get_amex_metric_calculated(y_true: np.array, y_pred: np.array) -> float:

    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return 0.5 * (g + d)

In [6]:
print(get_amex_metric_calculated(y_true=df_train.target, y_pred=df_train.target))

1.0000000000015785
